# 2D Backends

In the following example, first we will show the results with the default backend, Matplotlib, and explain the limitations. Then we will show what a different backend can do.

In [1]:
%matplotlib widget
from sympy import *
from sympy.plotting.plot import plot3d, plot3d_parametric_surface
from spb.plotly import PB
from spb.bokeh import BB
var("u, v, x, y")

(u, v, x, y)

In the above code cell we imported two backends. The following submodules are available:
* `bokeh`, from which we can import `BokehBackend` or its alias `BB`.
* `plotly`, from which we can import `PlotlyBackend` or its alias `PB`.
* `k3d`, from which we can import `K3DBackend` or its alias `KB`.
* `mayavi`, from which we can import `MayaviBackend` or its alias `MB`.

Only `BokehBackend` and `PlotlyBackend` support 2D plots.

In [ ]:
plot(sin(x), cos(x), log(x), legend=True)

The plot looks fine.
* In order to interact with it we have to use the buttons on the toolbar.
* If we move the cursor over the plot, we can see its coordinates. With this behaviour we can only get approximate coordinates if we move the cursor over a line.

Let's visualize the same plot with `PlotlyBackend`:

In [ ]:
plot(sin(x), cos(x), log(x), legend=True, backend=PB)

We can also interact with this plot using the top toolbar, however there are more natural ways:
* Click and drag to zoom into a rectangular selection.
* Move the cursor in the middle of the horizontal axis, click and drag to pan horizontally.
* Move the cursor in the middle of the vertical axis, click and drag to pan vertically.
* Move the cursor near the ends of the horizontal/vertical axis: click and drag to resize.
* Move the cursor over a line: a tooltip will show the coordinate of that point in the data series. Note that there is no interpolation between two consecutive points.
* Click over a label in the legend to hide/show that data series.

Let's now visualize the same plot with BokehBackend:

In [ ]:
plot(sin(x), cos(x), log(x), legend=True, backend=BB)

Here, we can:
* Click and drag to pan the plot around.
* Click and drag the axis to pan the plot only on one direction.
* Click the legend entries to hide/show the data series.
* Move the cursor over a line: a tooltip will show the coordinate of that point in the data series.
* Use the toolbar to change the tool, for example we can select the _Box Zoom_ to zoom into a rectangular region.

Note that, differently from Plotly, Bokeh doesn't support Latex!

In [4]:
p = plot(cos(x), sin(x), log(x), backend=PB, show=False, legend=True, title="Plotly - Latex Support",
        xlabel=r"$\xi$", ylabel=r"$\eta$")
p[0].label = "$f_{1}$"
p[1].label = "$f_{2}$"
p[2].label = "$f_{3}$"
p.show()

At the time of writing this, there might be problems when using [Plotly with Latex on Firefox](https://github.com/plotly/plotly.js/issues/5374).

In [2]:
p = plot(cos(x), sin(x), log(x), backend=BB, legend=True, title="Bokeh")

Loading BokehJS ...

Loading BokehJS ...